<a href="https://colab.research.google.com/github/tech-david/DSC570/blob/main/Topic1/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [2]:
iris = pd.read_csv('iris.data',
                     names=['sepal length',
                            'sepal width',
                            'petal length',
                            'petal width',
                            'class'])

In [3]:
iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
class Chi_Merge_Discretization:
    def __init__(self):
        print('Chi Merge Discretization')
    

    def get_intervals(cls, intervals, chi, min_chi):
        
        min_chi_index = np.where(chi == min_chi)[0][0]
        new_intervals = []
        do_skip = False
        is_done = False
        for interval in range(len(intervals)):
            if do_skip:
                do_skip = False
                continue
            if interval == min_chi_index and not is_done:
                j = intervals[interval] + intervals[interval+1]
                new_intervals.append([min(j), max(j)])
                do_skip = True
                is_done = True
            else:
                new_intervals.append(intervals[interval])
        return new_intervals        
    

    def chimerge(cls, data, colName, label, max_intervals):
        
        # Get features
        features = np.unique(data[colName])
        
        # Get classes
        classes = np.unique(data[label])
        
        # Initialize all values of classes to 0
        empty_count = {c: 0 for c in classes}
        intervals = [[features[i], features[i]] for i in range(len(features))]
        while len(intervals) > max_intervals:
            chi = []
            for i in range(len(intervals)-1):
                
                # left interval and right interval
                left_interval = data[data[colName].between(intervals[i][0], intervals[i][1])]
                right_interval = data[data[colName].between(intervals[i+1][0], intervals[i+1][1])]
                total = len(left_interval) + len(right_interval)
                
                # Creating contingency_table
                count_1 = np.array([j for i, j in {**empty_count, **Counter(left_interval[label])}.items()])
                count_2 = np.array([j for i, j in {**empty_count, **Counter(right_interval[label])}.items()])
                count_total = count_1 + count_2
                
                # Calculate E 
                expected_1 = count_total*sum(count_1)/total
                expected_2 = count_total*sum(count_2)/total
                # Get chi squared 
                chi_square = (count_1 - expected_1)**2/expected_1 + (count_2 - expected_2)**2/expected_2
                
                # Drop NaN values resulting from division of 0
                chi_square = np.nan_to_num(chi_square)
                # Append to chi array
                chi.append(sum(chi_square))
                
            # Get minimum chi value    
            min_chi = min(chi)
            
            intervals = cls.get_intervals(intervals, chi, min_chi)
        print('Min chi square: ' + str(min_chi))
        return intervals

In [9]:
if __name__ == '__main__':
    max_intervals = 4
    obj = Chi_Merge_Discretization()
    for colName in iris.columns[0:-1]:
        print('\n Intervals for', colName)
        intervals = obj.chimerge(iris, colName, iris.columns[-1], max_intervals)
        print(tabulate([[intervals]], tablefmt='pretty'))

Chi Merge Discretization

 Intervals for sepal length
Min chi square: 3.5950617283950614
+--------------------------------------------------+
| [[4.3, 5.4], [5.5, 5.7], [5.8, 7.0], [7.1, 7.9]] |
+--------------------------------------------------+

 Intervals for sepal width
Min chi square: 5.796
+--------------------------------------------------+
| [[2.0, 2.4], [2.5, 2.9], [3.0, 3.3], [3.4, 4.4]] |
+--------------------------------------------------+

 Intervals for petal length
Min chi square: 1.9444444444444442
+--------------------------------------------------+
| [[1.0, 1.9], [3.0, 4.7], [4.8, 5.1], [5.2, 6.9]] |
+--------------------------------------------------+

 Intervals for petal width
Min chi square: 2.8962962962962955
+--------------------------------------------------+
| [[0.1, 0.6], [1.0, 1.3], [1.4, 1.7], [1.8, 2.5]] |
+--------------------------------------------------+
